In [170]:

from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import pandas as pd
import os, json

endpoint="https://anildwaopenaiwestus.openai.azure.com/"
deployed_model = "gpt-4-turbo"

token_provider = get_bearer_token_provider(
            DefaultAzureCredential(),
            "https://cognitiveservices.azure.com/.default"
        )

client = AzureOpenAI(
   api_version="2023-09-01-preview",
   azure_deployment=deployed_model,
   azure_endpoint=endpoint,
   api_key="3661ab7c5e4d49669cbe4d1d96ce6c3a",
   #azure_ad_token_provider=token_provider,   
)


def getOpenAIResp(userQuery, systemMessage, deployed_model):
    completion = client.chat.completions.create(
            model=deployed_model,
            messages=[
                {
                    "role": "system",
                    "content": systemMessage
                },
                {
                    "role": "user",
                    "content": userQuery
                }
            ],
            temperature=0,
            max_tokens=4000,
            stream=False)
    # print(completion.choices[0].message.content)
    return completion.choices[0].message.content

def getFewShotExamples():
    examplesDir = os.path.join(".", "Data Sets Schema and Examples")

    fewshot_examples = ""
    for index, dir in enumerate(os.listdir(examplesDir)):
        # is it a directory
        if not os.path.isdir(os.path.join(examplesDir, dir)):
            continue

        if index < 3:
            #print(dir)
            #print(index)
            sql_query = open(os.path.join(examplesDir, dir, "dataset_mssql_query.sql"), "r").read()
            dataset_xml = open(os.path.join(examplesDir, dir, "dataset_xml.xml"), "r").read()
            
            fewshot_examples += f"""example {index}: \n\n 
            
            sql query: \n\n{sql_query}

            generated xml: \n\n{dataset_xml}

            """
    
    return fewshot_examples
            #print(sql_query)
            #print(dataset_xml)

task1_system_message = f"""

You are SQL to XML conversion expert. You will be given a SQL query and XML that is mapped to the SQL query.
You need to generate mapping rules from the given SQL query and XML structure so that the SQL query can be converted to XML.
Understand the SQL query and XML structure given below and generate the mapping rules. 



"""

task2_system_message = f"""
You are SQL to XML conversion expert. You will be given a SQL query and you need to convert it to XML. 
        Understand the SQL query and XML structure given below. 
        Generate the XML of the given SQL query that is as accurate as possible compared to the provided XML structure and XML elements in the XML example.
        Just provide the XML of the SQL query.
        Always generate complete XML without omitting any element.
        

        Examples:
"""
def getSystemMessage(system_message, context=None):
    system_message = f"""{system_message}
        

        {getFewShotExamples()}   

    Mapping rules:

    {context}
    """
    return system_message


In [102]:
def generate_embeddings(text, model="text-embedding-ada-002"):    
    try:
        with open('config.json') as f:
         embeddings_config = json.load(f)
        openai_client = AzureOpenAI(
            azure_endpoint=embeddings_config[0]["embedding"][0]["endpoint"],
            api_key=embeddings_config[0]["embedding"][0]["api-key"],
            #azure_endpoint=embedding_endpoint,
            #api_key=embedding_api_key,
            api_version="2023-05-15",
        )
        embeddings = openai_client.embeddings.create(input = text, model=embeddings_config[0]["embedding"][0]["model"]).data
        return embeddings
    except Exception as e:
        print("Error calling OpenAI:" + openai_client.base_url)
        print(e)
        raise


In [103]:
text = "hello world"
print(generate_embeddings(text))

[Embedding(embedding=[-0.006714438553899527, -0.03919780254364014, 0.03417709469795227, 0.02876320295035839, -0.024785956367850304, -0.042010609060525894, -0.030260341241955757, 0.049299705773591995, -0.013943045400083065, -0.01766320690512657, 0.015379693359136581, -0.026948489248752594, -0.02099018171429634, -0.02781047858297825, 0.00861232541501522, 0.03565911203622818, -0.053624771535396576, -0.0023515657521784306, 0.008839164860546589, 0.04802940413355827, 0.03711088374257088, -0.009270159527659416, -0.008740867488086224, 0.011402446776628494, 0.014109394513070583, -0.002185216872021556, -0.037564560770988464, 0.04539807140827179, 0.011228537186980247, -0.03962123394012451, 0.023424921557307243, -0.05063049495220184, 0.01199979055672884, -5.354939276003279e-05, 0.01606021076440811, 0.006151121109724045, 0.031999439001083374, 0.003408258082345128, -0.008597203530371189, -0.010570704005658627, -0.03738309070467949, -0.034479547291994095, 0.04999534413218498, 0.01938718371093273, -0.

In [104]:
from collections import namedtuple

metadata_schema = namedtuple("metadata_vectors", ["metadata_type", "metadata_typename", "metadatavector"])


metadata_vectors = []

In [129]:
import numpy as np
def process_metadata(mt_to_vectorize, mt):
    mt['metadatavector'] = generate_embeddings(json.dumps(mt_to_vectorize))[0].embedding
    return mt


def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    #print(a.shape)
    #print(b.shape)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [139]:
# Assuming generate_embeddings and other required functions are already defined
import json
import os
import concurrent.futures
import numpy as np

# Load the metadata JSON
with open(os.path.join(".", "Data Sets Schema and Examples","Metadata schema.json")) as metadata_schema:
    metadata_schema_json = json.load(metadata_schema)

# Using ThreadPoolExecutor to process metadata in parallel
metadata_vectors = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for metadata_key, metadata in metadata_schema_json.items():
        if isinstance(metadata, dict):
            for key, value in metadata.items():
                mt = {"metadata_type": metadata_key, "metadata_typename": key, "metadata_value" : value}
                futures.append(executor.submit(process_metadata, value, mt))
        elif isinstance(metadata, list):
            for item in metadata:
                mt = {"metadata_type": metadata_key, "metadata_typename": "ReferenceForms", "metadata_value" : item}
                futures.append(executor.submit(process_metadata, item, mt))

    for future in concurrent.futures.as_completed(futures):
        metadata_vectors.append(future.result())




In [142]:
metadata_vectors[100]['metadata_value']


{'DslTableId': 'Countries',
 'ColumnName': 'ISO3Code',
 'Type': 'SystemTable',
 'FieldId': 29,
 'SourceId': 4,
 'CanSelect': True,
 'CanSelectInFunctions': [],
 'CanSort': True,
 'CanFilter': True,
 'FilterIfAny': {'ValueType': 'Dictionary',
  'AvailableOperators': ['In', 'NotIn'],
  'LookupDatasource': {'Type': 'SystemTable',
   'DatasourceClassName': 'Iso3CodeEnumOptions'}},
 'DisplayName': 'ISO Code',
 'CanJoinOn': False}

In [145]:
# Load and embed the SQL query
testFile = os.path.join(".", "Data Sets Schema and Examples", "dataset_01", "dataset_mssql_query.sql")
sql_query_text = open(testFile, "r").read()
sql_query_vector = generate_embeddings(json.dumps({"query": sql_query_text}))

# Calculate cosine similarities and store with metadata information
similarities = []
for mt in metadata_vectors:
    #print(mt['metadata_value'])
    similarity = cosine_similarity(mt['metadatavector'], sql_query_vector[0].embedding)
    similarities.append((similarity, mt['metadata_type'], mt['metadata_typename'], mt['metadata_value']))

# Sort by similarity (highest first) and select top entries
similarities.sort(reverse=True, key=lambda x: x[0])
top_similarities = similarities[:5]  # Adjust the number to get the desired count of top entries

# Display the top similarities
for sim, mtype, mname, mvalue in top_similarities:
    print(f"Similarity: {sim}, Type: {mtype}, Name: {mname}, Value: {mvalue}")

with open('metadata_vectors.json', 'w') as f:
    json.dump(metadata_vectors, f)


Similarity: 0.6249215952537239, Type: Fields, Name: 354, Value: {'DslTableId': 'Requests', 'ColumnName': 'ClientName', 'Type': 'SystemTable', 'FieldId': 354, 'SourceId': 45, 'CanSelect': True, 'CanSelectInFunctions': [], 'CanSort': True, 'CanFilter': True, 'FilterIfAny': {'ValueType': 'String', 'AvailableOperators': ['Equals', 'NotEquals', 'IsNull', 'IsNotNull', 'Contains', 'NotContains', 'In', 'NotIn']}, 'DisplayName': 'Client Name', 'CanJoinOn': False}
Similarity: 0.6246721432236668, Type: Fields, Name: 741, Value: {'DslTableId': 'Requests', 'ColumnName': 'ClientName', 'Type': 'SystemTable', 'FieldId': 741, 'SourceId': 71, 'CanSelect': True, 'CanSelectInFunctions': [], 'CanSort': True, 'CanFilter': True, 'FilterIfAny': {'ValueType': 'String', 'AvailableOperators': ['Equals', 'NotEquals', 'IsNull', 'IsNotNull', 'Contains', 'NotContains', 'In', 'NotIn']}, 'DisplayName': 'Client Name', 'CanJoinOn': False}
Similarity: 0.6241581629700099, Type: Fields, Name: 712, Value: {'DslTableId': 'Re

In [146]:
metadata_vectors = json.loads(open('metadata_vectors.json', 'r').read())

In [151]:
def getMetadata(sqlQuery_parent_dir):
    testFile = os.path.join(".", "Data Sets Schema and Examples", sqlQuery_parent_dir, "dataset_mssql_query.sql")
    sql_query_text = open(testFile, "r").read()
    sql_query_vector = generate_embeddings(json.dumps({"query": sql_query_text}))
    for mt in metadata_vectors:
        #print(mt['metadata_value'])
        similarity = cosine_similarity(mt['metadatavector'], sql_query_vector[0].embedding)
        similarities.append((similarity, mt['metadata_type'], mt['metadata_typename'], mt['metadata_value']))
    similarities.sort(reverse=True, key=lambda x: x[0])
    top_similarities = similarities[:5]
    sqlMetadata = "SQL Query Metadata:\n"
    for sim, mtype, mname, mvalue in top_similarities:
        sqlMetadata += f" {json.dumps(mvalue)}\n"
        return sqlMetadata


In [152]:
getMetadata("dataset_01")

'SQL Query Metadata:\n {"DslTableId": "Requests", "ColumnName": "ClientName", "Type": "SystemTable", "FieldId": 354, "SourceId": 45, "CanSelect": true, "CanSelectInFunctions": [], "CanSort": true, "CanFilter": true, "FilterIfAny": {"ValueType": "String", "AvailableOperators": ["Equals", "NotEquals", "IsNull", "IsNotNull", "Contains", "NotContains", "In", "NotIn"]}, "DisplayName": "Client Name", "CanJoinOn": false}\n'

In [171]:
getSystemMessage(task1_system_message)

'\n\nYou are SQL to XML conversion expert. You will be given a SQL query and XML that is mapped to the SQL query.\nYou need to generate mapping rules from the given SQL query and XML structure so that the SQL query can be converted to XML.\nUnderstand the SQL query and XML structure given below and generate the mapping rules. \n\n\n\n\n        \n\n        example 1: \n\n \n            \n            sql query: \n\nSELECT\nTOP 500\n     [Clients].[ClientId] AS [0],\n     [Clients].[Name] AS [1],\n     [Clients].[Description] AS [2]\nFROM\n     [dbo].[Clients] AS [Clients]\n\nWHERE \n     [Clients].[Name] NOT LIKE \'\'%\'\' + @p__dsl__p0 + \'\'%\'\'\',N\'@p__dsl__p0 nvarchar(4)\',@p__dsl__p0=N\'name\'\n\n            generated xml: \n\n<QueryIntegrations xmlns:d2p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.Integrations">\n    <d2p1:QueryIntegration>\n      <Id xmlns="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model">1178</Id>\n      <SchemaVersion xmlns="http://

In [158]:
def getXMLSchema():
    with open(os.path.join(".", "Data Sets Schema and Examples", "xml_schema.xml")) as xml_schema  :
        return xml_schema.read()

In [162]:
mapping_rules = getOpenAIResp("What are the mapping rules?", getSystemMessage(task1_system_message), deployed_model)

mapping_rules += f"""\n\n

### DataSetFilterCondition and FormAnswerConditions
For XML nodes with DataSetFilterCondition and FormAnswerConditions, fill these nodes only if you need to based on the SQL query.
Use only 
Generate full XML of the given SQL query. Do not miss or omit any element or attribute.\n\n

### Use the below XML Schema to generate the XML where needed.\n\n
{getXMLSchema()}

### Use the SQL Query Metadata to generate the XML where needed.


"""

In [164]:
print(mapping_rules)


with open(os.path.join(".", "Data Sets Schema and Examples", "mapping_rules.txt"), 'w') as f:
    f.write(mapping_rules)

To generate the mapping rules from the given SQL query to the XML structure, we need to analyze the SQL query components and map them to the corresponding XML elements. Here are the mapping rules based on the provided examples:

### Common Mapping Rules for Both Examples:

1. **Query Limit:**
   - SQL: `TOP 500`
   - XML: `<d5p1:QueryLimit>500</d5p1:QueryLimit>`

2. **Source Table:**
   - SQL: `[dbo].[Clients] AS [Clients]`
   - XML: 
     ```xml
     <d5p1:Source i:type="d6p1:SystemTable">
       <d6p1:Alias>Clients</d6p1:Alias>
       <d6p1:TableId>Clients</d6p1:TableId>
     </d5p1:Source>
     ```

3. **Select Fields:**
   - SQL: `[Clients].[ClientId] AS [0]`, `[Clients].[Name] AS [1]`, etc.
   - XML: Each selected field in SQL is represented as a `<d5p1:SelectExpression>` with a `<d5p1:ColumnName>` and `<d5p1:Expression>` in XML.

### Specific Mapping Rules for Example 1:

1. **Condition (WHERE Clause):**
   - SQL: `[Clients].[Name] NOT LIKE '%...%'`
   - XML: 
     ```xml
     <d

In [172]:
testFilesDir = os.path.join(".", "Data Sets Schema and Examples")
system_message = getSystemMessage(task2_system_message, mapping_rules)
print(system_message)


You are SQL to XML conversion expert. You will be given a SQL query and you need to convert it to XML. 
        Understand the SQL query and XML structure given below. 
        Generate the XML of the given SQL query that is as accurate as possible compared to the provided XML structure and XML elements in the XML example.
        Just provide the XML of the SQL query.
        Always generate complete XML without omitting any element.
        

        Examples:

        

        example 1: 

 
            
            sql query: 

SELECT
TOP 500
     [Clients].[ClientId] AS [0],
     [Clients].[Name] AS [1],
     [Clients].[Description] AS [2]
FROM
     [dbo].[Clients] AS [Clients]

WHERE 
     [Clients].[Name] NOT LIKE ''%'' + @p__dsl__p0 + ''%''',N'@p__dsl__p0 nvarchar(4)',@p__dsl__p0=N'name'

            generated xml: 

<QueryIntegrations xmlns:d2p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.Integrations">
    <d2p1:QueryIntegration>
      <Id xmlns="http://sche

In [173]:

for index, dir in enumerate(os.listdir(testFilesDir)):
    if not os.path.isdir(os.path.join(testFilesDir, dir)):
            continue

    if index > 2:
        #print(dir)
        sql_query = open(os.path.join(testFilesDir, dir, "dataset_mssql_query.sql"), "r").read()
        sql_query_metadata = getMetadata(dir)
        #print(sql_query)
        user_query = f"""Convert the following SQL query to XML Just provide the XML and nothing else.\n
            {sql_query_metadata}
            SQL Query: \n\n {sql_query}\n\n Generated XML: \n\n"""
        generated_xml = getOpenAIResp(user_query, system_message, deployed_model)
        print(generated_xml)
        with open(os.path.join(testFilesDir, dir, "generated_xml.xml"), "w") as f:
            f.write(generated_xml)
          


```xml
<QueryIntegrations xmlns:d2p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.Integrations">
  <d2p1:QueryIntegration>
    <Id xmlns="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model">1180</Id>
    <SchemaVersion xmlns="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model">3700</SchemaVersion>
    <d2p1:Active>true</d2p1:Active>
    <d2p1:DatasourceId>2</d2p1:DatasourceId>
    <d2p1:Draft>false</d2p1:Draft>
    <d2p1:IgnoreClientMatterSecurity>false</d2p1:IgnoreClientMatterSecurity>
    <d2p1:IsIntegrationTypeUserSelected>false</d2p1:IsIntegrationTypeUserSelected>
    <d2p1:Name>Dataset_03</d2p1:Name>
    <d2p1:QueryBuilder i:type="d2p1:OpenLinkQueryBuilder">
      <d2p1:DataSetSettings xmlns:d5p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets" i:type="d5p1:DataSetSettings">
        <d5p1:AdvancedExpression i:nil="true" />
        <d5p1:AggregationFunction xmlns:d6p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.Dat

In [3]:
getOpenAIResp("tell me a joke",getSystemMessage(None), deployed_model)

"Why don't skeletons fight each other?\n\nThey don't have the guts."

In [27]:
# read the file
import os

with open(os.path.join(".", "query 3.txt"), "r") as file:
    sql_txt = file.read()
    #print(sql_txt)
    user_query = f"sql to convert: {sql_txt}"
    llm_response = getOpenAIResp(user_query, getSystemMessage(""), 'gpt-4-turbo')

In [28]:
from IPython.display import Markdown


with open(os.path.join(".", "generated_xml", "gen dataset 3.txt"), "w") as file:
    file.write(llm_response)


Markdown(llm_response)

```xml
<QueryIntegration xmlns:i="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.Integrations">
    <Id xmlns="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model">0</Id> <!-- Assuming Id as 0 since it's not provided -->
    <SchemaVersion xmlns="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model">0</SchemaVersion> <!-- Assuming SchemaVersion as 0 since it's not provided -->
    <Active>true</Active>
    <DatasourceId>0</DatasourceId> <!-- Assuming DatasourceId as 0 since it's not provided -->
    <Draft>false</Draft>
    <IgnoreClientMatterSecurity>false</IgnoreClientMatterSecurity>
    <IsIntegrationTypeUserSelected>false</IsIntegrationTypeUserSelected>
    <Name>Retrieve Attachments by Request Id</Name> <!-- Assuming a name for the query -->
    <QueryBuilder i:type="OpenLinkQueryBuilder">
        <DataSetSettings xmlns:d3p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets" i:type="d3p1:DataSetSettings">
            <d3p1:AdvancedExpression i:nil="true" />
            <d3p1:AggregationFunction xmlns:d4p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets.AggregationFunctions" i:nil="true" />
            <d3p1:CollectionType>All</d3p1:CollectionType>
            <d3p1:ComputedFieldFlags xmlns:d4p1="http://schemas.microsoft.com/2003/10/Serialization/Arrays" i:nil="true" />
            <d3p1:DataSourceId i:nil="true" />
            <d3p1:ExternalParameters xmlns:d4p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets.ExternalParameters" />
            <d3p1:FieldColumnTitles xmlns:d4p1="http://schemas.microsoft.com/2003/10/Serialization/Arrays" i:nil="true" />
            <d3p1:FieldNames xmlns:d4p1="http://schemas.microsoft.com/2003/10/Serialization/Arrays" />
            <d3p1:OperationId i:nil="true" />
            <d3p1:ParameterExpressions />
            <d3p1:ReturnDistinctRows>false</d3p1:ReturnDistinctRows>
        </DataSetSettings>
        <DslQuerySettings xmlns:d3p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets.DSL.QuerySettings">
            <d3p1:AggregationFunction xmlns:d4p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets.AggregationFunctions" i:nil="true" />
            <d3p1:ComplexSourceQuestion xmlns:d4p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets.DSL.QuerySettings.FormAnswers" i:nil="true" />
            <d3p1:Conditions xmlns:d4p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets.DSL.QuerySettings.Conditions">
                <d4p1:AdvancedExpression i:nil="true" />
                <d4p1:CollectionType>All</d4p1:CollectionType>
                <d4p1:Conditions>
                    <d4p1:DataSetFilterCondition>
                        <d4p1:Left i:type="d3p1:EncodedSystemTableField">
                            <d3p1:Value>#:#system#:#Requests#:#Requests#:#Id#:#</d3p1:Value>
                        </d4p1:Left>
                        <d4p1:Operator>Equals</d4p1:Operator>
                        <d4p1:Right i:type="d4p1:ParameterOperand">
                            <d4p1:DefaultValue i:nil="true" />
                            <d4p1:External>true</d4p1:External>
                            <d4p1:Name>p0</d4p1:Name>
                            <d4p1:ValueType>Int</d4p1:ValueType>
                        </d4p1:Right>
                    </d4p1:DataSetFilterCondition>
                    <d4p1:DataSetFilterCondition>
                        <d4p1:Left i:type="d3p1:EncodedSystemTableField">
                            <d3p1:Value>#:#system#:#Requests#:#Requests#:#Deleted#:#</d3p1:Value>
                        </d4p1:Left>
                        <d4p1:Operator>Equals</d4p1:Operator>
                        <d4p1:Right i:type="d4p1:LiteralOperand">
                            <d4p1:Value>0</d4p1:Value>
                        </d4p1:Right>
                    </d4p1:DataSetFilterCondition>
                </d4p1:Conditions>
            </d3p1:Conditions>
            <d3p1:Description i:nil="true" />
            <d3p1:Distinct>false</d3p1:Distinct>
            <d3p1:EnforceSecurity>true</d3p1:EnforceSecurity>
            <d3p1:ExternalParameters i:nil="true" />
            <d3p1:FormAnswerConditions xmlns:d4p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets.DSL.QuerySettings.Conditions" i:nil="true" />
            <d3p1:Joined xmlns:d4p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets.DSL.QuerySettings.Source">
                <d4p1:JoinedTable>
                    <d4p1:JoinCondition>
                        <d4p1:Left i:type="d3p1:EncodedSystemTableField">
                            <d3p1:Value>#:#system#:#Attachments#:#Attachments#:#RootComponentIdFk#:#</d3p1:Value>
                        </d4p1:Left>
                        <d4p1:Operator>Equals</d4p1:Operator>
                        <d4p1:Right i:type="d3p1:EncodedSystemTableField">
                            <d3p1:Value>#:#system#:#Requests#:#Requests#:#ComponentIdFk#:#</d3p1:Value>
                        </d4p1:Right>
                    </d4p1:JoinCondition>
                    <d4p1:Table i:type="d4p1:SystemTable">
                        <d4p1:Alias>Attachments</d4p1:Alias>
                        <d4p1:TableId>Attachments</d4p1:TableId>
                    </d4p1:Table>
                </d4p1:JoinedTable>
            </d3p1:Joined>
            <d3p1:QueryLimit>500</d3p1:QueryLimit>
            <d3p1:ReferenceForm xmlns:d4p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets.DSL.QuerySettings.FormAnswers" i:nil="true" />
            <d3p1:RuntimeParameters i:nil="true" />
            <d3p1:SelectFields>
                <d3p1:SelectExpression>
                    <d3p1:ColumnName>Key</d3p1:ColumnName>
                    <d3p1:Expression>[#:#system#:#Attachments#:#Attachments#:#Id#:#]</d3p1:Expression>
                </d3p1:SelectExpression>
            </d3p1:SelectFields>
            <d3p1:SortFields i:nil="true" />
            <d3p1:Source xmlns:d4p1="http://schemas.datacontract.org/2004/07/IntApp.Wilco.Model.DataSets.DSL.QuerySettings.Source" i:type="d4p1:SystemTable">
                <d4p1:Alias>Requests</d4p1:Alias>
                <d4p1:TableId>Requests</d4p1:TableId>
            </d3p1:Source>
        </DslQuerySettings>
        <RunAsDSLQuery>true</RunAsDSLQuery>
    </QueryBuilder>
    <QueryIntegrationObjectType>Attachment</QueryIntegrationObjectType> <!-- Assuming the object type as Attachment -->
    <QueryIntegrationType>KeyValue</QueryIntegrationType>
    <useDeletedFlagForParties>true</useDeletedFlagForParties>
</QueryIntegration>
```